In [ ]:
HOST = "demo2.verta.ai"
PORT = "6244"

# Logistic Regression with Grid Search (scikit-learn)

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import os, sys
import itertools
import time
from multiprocessing import Pool

import numpy as np

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

---

# Log Workflow

## Prepare Data

In [ ]:
train_data = np.genfromtxt("train-data.csv", delimiter=',', skip_header=1)
X_train = train_data[:,:-1]
y_train = train_data[:, -1]

## Prepare Hyperparameters

In [ ]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Instantiate Client

In [ ]:
from verta import ModelDBClient

client = ModelDBClient(HOST, PORT)
proj = client.set_project("Project Name")
expt = client.set_experiment("Experiment Name")

## Run Validation

In [ ]:
def run_experiment(hyperparams):
    # create object to track experiment run
    run = client.set_experiment_run()
    
    # log data
    run.log_dataset("train_data", open("train-data.csv", 'rb'))
    
    # create validation split
    (X_val_train, X_val_test,
     y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                                 test_size=0.2,
                                                                 shuffle=True)
    
    # log hyperparameters
    run.log_hyperparameters(**hyperparams)
    print(hyperparams, end=' ')
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    run.log_metric("val_acc", val_acc)
    print("Validation accuracy: {:.4f}".format(val_acc))
    
    # save and log model
    run.log_deployment_model(model, "requirements.txt", "model_api.json")
    
with Pool() as pool:
    pool.map(run_experiment, hyperparam_sets)

---

# Make Live Predictions

In [ ]:
model_id = 

## Prepare Data

In [ ]:
test_data = np.genfromtxt("test-data.csv", delimiter=',', skip_header=1)
X_test = test_data[:,:-1]

## Load Deployed Model

In [ ]:
from verta.modeldbclient import ExperimentRun

deployed_model = ExperimentRun(client._auth, client._socket,
                               _expt_run_id=model_id)

## Query Deployed Model

In [ ]:
for x in itertools.cycle(X_test):
    deployed_model.predict(x)

---